In [35]:
import pickle

with open("../api_key.pkl", "rb") as f:
   API_KEY=pickle.load(f)

# print(API_KEY)

MODEL_ID="ali/qwen-turbo-latest"

In [100]:
import csv
import datetime

def log(s: str) -> None:
    """
    把一条日志追加到 log.csv。
    第一列：当前时间（ISO 8601，带毫秒）
    第二列：s 本身（自动处理里面的逗号、引号、换行）
    文件不存在会自动创建，存在则追加。
    """
    # 构造一行
    now=datetime.datetime.now()
    row = [f"{now.year}{now.month:02}{now.day:02} {now.hour:02}{now.minute:02}{now.second:02}.{now.microsecond/1000:03.0f}", s]

    # 以追加模式打开，newline='' 防止 Windows 多空行
    with open('log.csv', 'a', encoding='utf-8', newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
        # 如果文件刚创建，可写表头（按需）
        if f.tell() == 0:
            writer.writerow(['timestamp', 'message'])
        writer.writerow(row)


# 简单测试
if __name__ == '__main__':
    log("test")

In [63]:
import http.client
import json
import time

conn = http.client.HTTPSConnection("router.shengsuanyun.com")
HEADERS = {
   'HTTP-Referer': 'https://www.postman.com',
   'X-Title': 'Postman',
   'Authorization': API_KEY,
   'Content-Type': 'application/json'
}


In [ ]:
payload = json.dumps({
   "model": MODEL_ID,
   "stream": True,
   "messages": [
      {
         "role": "user",
         "content": "讲个笑话"
      },
      {
         "role": "assistant",
         "content": "cong qian you zuo shan"
      },
      {
         "role": "user",
         "content": "继续"
      }
   ],
   "stream_options": {
      "include_usage": True
   }
})
start_time = time.time()
conn.request("POST", "/api/v1/chat/completions", payload, HEADERS)
elapsed_time = time.time()-start_time
print(f"executed in {elapsed_time:.4f} seconds")
res = conn.getresponse()

for line in res:
    line = line.decode('utf-8').strip()
    try:
        json_str = line[6:]
        obj = json.loads(json_str)
        content = obj["choices"][0]["delta"]["content"]
        if content != None:
            print(content,end="")
        if "usage" in obj:
            print("\n\nusage:",obj["usage"])
    except:
        pass
    if line != "":
        log(line)


In [65]:
payload = json.dumps({
   "model": MODEL_ID,
   "stream": False,
   "messages": [
      {
         "role": "system",
         "content": """阿强是某 AI 平台的运维兼客服负责人。通过知识库，他维护了一套自动回复系统，但每当底层模型 API 出现波动或宕机时，用户依然会收到“千篇一律”的回答，而不知道系统真实的健康状况。
阿强希望构建一个“具备感知能力”的智能客服。它不仅能基于文档回答业务问题，还能实时连接 Uptime Kuma 监控系统。当 API 报错时，它能自动通过 Webhook 通知飞书运维群，并在 Apifox 中记录故障文档；当用户询问“今天系统稳定吗”时，它能基于监控历史给出真实的回答，而不是瞎编。"""
      },
      {
         "role": "user",
         "content": "讲个笑话"
      },
      {
         "role": "assistant",
         "content": "从前有座山"
      },
      {
         "role": "user",
         "content": "继续"
      }
   ],
   "stream_options": {
      "include_usage": True
   }
})
start_time = time.time()

conn.request("POST", "/api/v1/chat/completions", payload, HEADERS)
res = conn.getresponse()
obj = json.loads(res.read().decode('utf-8'))
elapsed_time = time.time()-start_time
content=obj["choices"][0]["message"]["content"]

print(f"executed in {elapsed_time:.4f} seconds")
print(content,"\n\nusage:",obj["usage"])
log(obj)

executed in 1.4925 seconds
山里有座庙，庙里有个老和尚在讲笑话。  
小和尚问：“师父，您讲的笑话为什么都这么冷？”  
老和尚说：“因为……我最近没开暖气。”  

小和尚恍然大悟：“原来不是笑话冷，是系统卡顿了！”  

这时，监控系统突然报警——  
“API 服务宕机！请立即处理！”  

老和尚淡定地掏出手机，发了个飞书消息：  
“兄弟们，别慌，我刚把‘冷笑话’升级成‘热故障通知’了。”  

——这就是为什么我们不靠 AI 写笑话，而是让它感知真实世界。  
（现在，你听到的不是笑话，是系统心跳 ❤️） 

usage: {'prompt_tokens': 175, 'completion_tokens': 155, 'total_tokens': 330}


In [13]:
del conn